###  Read the data


In [1]:
# Read the data into dataframes
import pandas as pd

train_df = pd.read_csv('training.csv')
print ("training set has ", train_df.shape)
test_df = pd.read_csv('test.csv')
print ("test set has ", test_df.shape)
ntest = test_df.shape[0]

print ("remove one column from the training set")
print ("merge training set and test set to all_data")

Ytrain = train_df['IsBadBuy']
train_df = train_df.drop(columns=['IsBadBuy'])

# Put all data together so we can wrangle all the data at the same time
all_data = pd.concat([train_df, test_df], ignore_index=True)
print ("all_data has ", all_data.shape)


training set has  (72983, 34)
test set has  (48707, 33)
remove one column from the training set
merge training set and test set to all_data
all_data has  (121690, 33)


In [2]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121690 entries, 0 to 121689
Data columns (total 33 columns):
RefId                                121690 non-null int64
PurchDate                            121690 non-null object
Auction                              121690 non-null object
VehYear                              121690 non-null int64
VehicleAge                           121690 non-null int64
Make                                 121690 non-null object
Model                                121690 non-null object
Trim                                 117780 non-null object
SubModel                             121677 non-null object
Color                                121678 non-null object
Transmission                         121678 non-null object
WheelTypeID                          116333 non-null float64
WheelType                            116328 non-null object
VehOdo                               121690 non-null int64
Nationality                          121678 non-null

###  Drop the unimportant features

There are too many missing data in `PRIMEUNIT` and `AUCGUART`, so we can just remove these features.
Also `RefId` has nothing to do but only the ID number that can be removed as well.

In [3]:
all_data.drop(labels='AUCGUART', axis='columns', inplace=True)
all_data.drop(labels='PRIMEUNIT', axis='columns', inplace=True)
all_data.drop(labels='RefId', axis='columns', inplace=True)

### Deal with duplicated data

In [4]:
all_data[all_data.duplicated(keep=False)].index

Int64Index([90880, 90886], dtype='int64')

There is not duplicated data, that is good

##  Numeric Data
###  Data histogram

###  Visualize the data


In [ ]:
import numpy as np

data_object = list(all_data.select_dtypes(include=[np.object]))
print (data_object)

import seaborn as sns
g=sns.catplot("Make", data = all_data, hue = "Make", kind = 'count', height=5, aspect=1.8,palette="winter",legend_out=False)
g.set_axis_labels("Make", "Count")
#sns.plt.title("Half time Score statistic")
#g.set(ylim=(0, 30))
#g.despine(left=True)  
#rc={'axes.labelsize': 16, 'font.size': 22.0, 'legend.fontsize': 16, 'axes.titlesize': 20}
#sns.set(rc=rc)

['PurchDate', 'Auction', 'Make', 'Model', 'Trim', 'SubModel', 'Color', 'Transmission', 'WheelType', 'Nationality', 'Size', 'TopThreeAmericanName', 'VNST']


### Deal with missing values

<p>Let's take a look at what missing values we'll have to handle.</p> For the numeric variable, replace null by median and for the category variable, most popular value is used to fill null

In [ ]:
all_isnull = all_data.columns[all_data.isnull().any()]
for col in all_isnull:
    try:
        all_data[col].fillna(all_data[col].mode()[0], inplace=True)
    except:
        all_data[col].fillna(all_data[col].median(), inplace=True)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121690 entries, 0 to 121689
Data columns (total 30 columns):
PurchDate                            121690 non-null object
Auction                              121690 non-null object
VehYear                              121690 non-null int64
VehicleAge                           121690 non-null int64
Make                                 121690 non-null object
Model                                121690 non-null object
Trim                                 121690 non-null object
SubModel                             121690 non-null object
Color                                121690 non-null object
Transmission                         121690 non-null object
WheelTypeID                          121690 non-null float64
WheelType                            121690 non-null object
VehOdo                               121690 non-null int64
Nationality                          121690 non-null object
Size                                 121690 non-nul

### Deal with category variable


In [ ]:
import category_encoders as ce

import numpy as np

nonList = list(all_data.select_dtypes(include=[np.object]))
print (nonList)

all_data2 = all_data.copy()

for this_label in nonList:
    
    encoder = ce.BinaryEncoder(cols=[this_label])
    
    all_data2 = encoder.fit_transform(all_data2)    
    
    print (this_label, len(all_data[this_label].unique()))
    if (len(all_data[this_label].unique()) < 40):
        print (all_data[this_label].unique())
    

['PurchDate', 'Auction', 'Make', 'Model', 'Trim', 'SubModel', 'Color', 'Transmission', 'WheelType', 'Nationality', 'Size', 'TopThreeAmericanName', 'VNST']
PurchDate 519
Auction 3
['ADESA' 'OTHER' 'MANHEIM']
Make 33
['MAZDA' 'DODGE' 'FORD' 'MITSUBISHI' 'KIA' 'GMC' 'NISSAN' 'CHEVROLET'
 'SATURN' 'CHRYSLER' 'MERCURY' 'HYUNDAI' 'TOYOTA' 'PONTIAC' 'SUZUKI'
 'JEEP' 'HONDA' 'OLDSMOBILE' 'BUICK' 'SCION' 'VOLKSWAGEN' 'ISUZU'
 'LINCOLN' 'MINI' 'SUBARU' 'CADILLAC' 'VOLVO' 'INFINITI' 'PLYMOUTH'
 'LEXUS' 'ACURA' 'TOYOTA SCION' 'HUMMER']
Model 1130
Trim 137
SubModel 933
Color 17
['RED' 'WHITE' 'MAROON' 'SILVER' 'BLACK' 'GOLD' 'GREY' 'BLUE' 'BEIGE'
 'PURPLE' 'ORANGE' 'GREEN' 'BROWN' 'YELLOW' 'NOT AVAIL' 'OTHER' 'PINK']
Transmission 3
['AUTO' 'MANUAL' 'Manual']
WheelType 3
['Alloy' 'Covers' 'Special']
Nationality 4
['OTHER ASIAN' 'AMERICAN' 'TOP LINE ASIAN' 'OTHER']
Size 12
['MEDIUM' 'LARGE TRUCK' 'COMPACT' 'LARGE' 'VAN' 'MEDIUM SUV' 'LARGE SUV'
 'SPECIALTY' 'SPORTS' 'CROSSOVER' 'SMALL SUV' 'SMALL TRU

###  Diving in (machine learning)

<p>Now that the data has been cleaned, we can try to find a model that works well for making our predictions. We'll also load in some classifiers which we will compare.</p>

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier

Xtrain = all_data2.head(len(Ytrain))

RFC = RandomForestClassifier()
Ada = AdaBoostClassifier()
KNN = KNeighborsClassifier()
classifiers = [RFC, Ada, KNN]
clf_names = ['Random Forest', 'AdaBoost', 'K Nearest Neighbors']

<p>For this analysis, we'll only be comparing across three classifiers: Random Forest, AdaBoost, and K Nearest Neighbors. For more information on other potential (or more complicated) classifiers I invite you to check out the other kernels posted by those who top the leaderboards for this competition.</p>

<p>For each of these classifiers, we'll want to make sure we create the models with the optimal parameters. We can do this with a Grid Search. We define the set of parameters we want to scan for each type of classifier, and then run our grid searches.</p>

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

# Use kfold as our cross validation
kfold = StratifiedKFold(n_splits=4)

# Set grid search parameter settings
rfc_param_grid = {'max_depth': [None],
                 'max_features': [1],
                 'min_samples_split': [2],
                 'min_samples_leaf': [1],
                 'bootstrap': [False],
                 'n_estimators': [100],
                 'criterion': ['gini']}
ada_param_grid = {'n_estimators': [20],
                 'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1, 10]}
knn_param_grid = {'n_neighbors': [5],
                  'weights': ['uniform', 'distance'],
                 'leaf_size': [5]}
param_grids = [rfc_param_grid, ada_param_grid, knn_param_grid]

# Perform grid searches to get estimators with the optimal settings
grid_searches = []
for i in range(len(classifiers)):
    grid_searches.append(GridSearchCV(estimator=classifiers[i], param_grid=param_grids[i], 
                                      n_jobs=4, cv=kfold, verbose=1))

<p>We'll now want to see the training scores for each of our models and determine which one works the best. We'll fit each model to our training set and add the best scores from each to a list.</p>

In [ ]:
# Train the models
best_scores = []
for i in range(len(grid_searches)):
    grid_searches[i].fit(Xtrain, Ytrain)
    best_scores.append(grid_searches[i].best_score_)

Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   40.2s remaining:   40.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   40.8s finished


Fitting 4 folds for each of 6 candidates, totalling 24 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


<p>Let's see the best scores for each classifier.</p>

In [ ]:
# Best scores
for i in range(len(best_scores)):
    print(clf_names[i] + ": " + str(best_scores[i]))

<p>Based on these training scores, it makes the most sense to use the Random Forest Classifier to make the predictions. We'll predict on the test set, and then write the predictions to a csv file for submission.</p>

In [ ]:
# Make predictions
Xtest = all_data2.tail(ntest)
#Xtest = test_df.drop(columns='RefId', axis='columns')
predictions = grid_searches[0].predict(Xtest)

# Write predictions to output csv
pred_df = pd.DataFrame({'RefId': test_df['RefId'],
                        'IsBadBuy': predictions})
pred_df.to_csv('predictions.csv', index=False)

print("Done writing to csv")